# Case study

## Extract companies' financial informations.

# Data extraction method

To retrieve the required informations I decided to use edgartools, a Python library with methods that allow to directly send API requests and retrieve the data.

The 'Edgartools' library converts the filings in data object, offering different methods to access specific parts of each filing.

Overall it allows for a faster and easier data retrievial compared to other available methods, allowing to directly convert the filings in dataframes, and to extract directly data of interest.

# Data extraction process

After creating an empty dataframe to fill with the retrieved companies' financial informations, I created a for loop to retrieve the data for 1000 companies.

# Steps:

- sending an api request for each company through the company 'CIK'.
- accessing the company information (address, company name, industry description) as well as the 10-K of the latest 5 years.
- accessing the financial statement of each 10-K, extracting the company revenue and the corresponding time value (year).
- adding the collected variables as a new row to the dataframe, with the informations about the company, year of financial statement, revenue, address, industry standard.

# Final step

After retrieving all the informations, some adjustment need to be made to get the correct format for 'geonameen'. Due to some conflict between dependencies, I finished this task in a separate notebook named: '2.Cleaning'.

In [ ]:
pip install edgartools

In [2]:
import pandas as pd
from edgar import *
import time

In [3]:
us_states = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA',
             'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
             'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY',
             'NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX',
             'UT','VT','VA','WA','WV','WI','WY']


In [4]:
set_identity("Sara Chiarelli sarachiarelli@outlook.it") # required to access Edgar  API

In [ ]:
filings = get_filings(2024, form= "10-K") # to get the latest companies' filings

In [6]:
df = filings.to_pandas() # converting the filings data in a pandas dataframe to access each row

In [11]:
df_companies = pd.DataFrame(columns=["timevalue", "companyname", "industryclassification", "Country", "revenue", "revenue_unit"])

In [12]:
i = 0
while len(df_companies["companyname"].unique()) <= 999: # the loop will stop when the data will be retrieved for 1000 unique companies (starting to count from 0)
  try:
    cik = df.loc[i,"cik"]
    company = Company(f"{cik}")
    name = company.name
    address = str(company.business_address.state_or_country_desc) # this return the state abbreviation for U.S. states, so to get the country I check if the address is in the list of U.S. states abbreviation.
    if address in us_states:
      country = "United States"
    else:
      country = address # for non U-S companies, the address returns a non standard description of the country, that I will standardize in the notebook '2.Cleaning'
    sic = company.sic_description
    latest_10k = Company("0001652539").get_filings(form="10-K").latest(5)
    for tenk in latest_10k:
      xbrl_data = tenk.xbrl()
      x = xbrl_data.instance.query_facts(concept="us-gaap:Revenues")
      revenue =  x.loc[[0],["value"]]
      revenue_unit = x.loc[[0],["units"]]
      timevalue= x.loc[[0],"start_date"]
      new_row = pd.DataFrame({
          "timevalue": [str(timevalue).split("-")[0].split(" ")[-1]],
          "companyname": [name],
          "industryclassification": [sic],
          "Country":[country],
          "revenue" : [str(revenue).split(" ")[-1]],
          "revenue_unit" : [str(revenue_unit).split(" ")[-1]]
          })
      df_companies = pd.concat([df_companies, new_row], ignore_index=True)

      i +=1

      # Respect rate limit: sleep for 0.1 seconds (10 requests per second)
      time.sleep(0.1)  # Add a 0.1-second delay between requests
  except:
    continue


In [13]:
len(df_companies["companyname"].unique())

1000

In [14]:
df_companies

,timevalue,companyname,industryclassification,Country,revenue,revenue_unit
0,2023,DAILY JOURNAL CORP,Newspapers: Publishing or Publishing & Printing,United States,610461,USD
1,2022,DAILY JOURNAL CORP,Newspapers: Publishing or Publishing & Printing,United States,589035,USD
2,2021,DAILY JOURNAL CORP,Newspapers: Publishing or Publishing & Printing,United States,754334,USD
3,2020,DAILY JOURNAL CORP,Newspapers: Publishing or Publishing & Printing,United States,1604606,USD
4,2019,DAILY JOURNAL CORP,Newspapers: Publishing or Publishing & Printing,United States,1384193,USD
...,...,...,...,...,...,...
5140,2023,FOSTER L B CO,Wholesale-Metals Service Centers & of fices,United States,610461,USD
5141,2022,FOSTER L B CO,Wholesale-Metals Service Centers & of fices,United States,589035,USD
5142,2021,FOSTER L B CO,Wholesale-Metals Service Centers & of fices,United States,754334,USD
5143,2020,FOSTER L B CO,Wholesale-Metals Service Centers & of fices,United States,1604606,USD


In [15]:
df_companies.to_excel("Case_Study_to_clean.xlsx")

In [16]:
from google.colab import files
files.download('Case_Study_to_clean.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>